In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import math

from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
DATA_DIR = Path('.', 'data', 'final', 'public')
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

In [90]:
del presotere

In [91]:
class presotere(TransformerMixin):
    
    def __init__(self,caso):
        self.lencoder_col=list([])
        self.object_cols=list([])
        self.number_cols = list([])
        self.lencoder_col=list([])
        self._caso=caso
        self.legeo = LabelEncoder()
        self.lencoder= list([])
        self._sns_data=None
        self._y=None
        self._numeric_transformer =[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())]     
        self._categorical_transformer = [
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))]       
        
    def inicializamodelo(self,caso):

        if caso == 1:
            self.legeo = LabelEncoder()
            self.lencoder= list([])
            print("Inicializa LabelEncoder caso 1")
        if caso == 2:
            self.legeo = LabelEncoder()
            self.lencoder= OneHotEncoder(handle_unknown='ignore', sparse=False)   
            print("Inicializa OneHotEncoder caso 2")
        
        
    def transform(self,X,y=None, **kwargs): 
        if self._caso == 1:
            contador=0
            self._sns_data=X[self.number_cols].copy()
            for col in X[self.object_cols].columns:
                self.lencoder.append(LabelEncoder())
                self.lencoder[contador].fit(X[col])
                self._sns_data[col]=self.lencoder[contador].transform(X[col])  
                contador=contador+1
                
                
        if self._caso == 2:  
            print("Transforma caso 2")
            nada = None
            self._sns_data=None
            print("Transforma caso 2.1")
            nada = self.lencoder.fit_transform(X[self.object_cols])
            co1c=0
            self.lencoder_col=list([])
            print("Transforma caso 2.2")
            for co1 in self.lencoder.categories_:
                for co2 in co1:
                    self.lencoder_col.append(self.object_cols[co1c]+"_"+co2)
                co1c=co1c+1
            print("Transforma caso 2.3",nada.shape,len(self.lencoder_col)) 
            objedf=pd.DataFrame(nada,columns=self.lencoder_col,
                                         index=X[self.object_cols].index.tolist())
            print("Transforma caso 2.4",objedf.shape) 
            self._sns_data=pd.concat([X[self.number_cols].copy(),objedf],axis=1)  
            print("NEcoder",self._sns_data.shape)
#            self._sns_data.to_csv(DATA_DIR / 'self._sns_data_05.csv')            
#            self._sns_data=self._sns_data.drop(self.object_cols,axis=1)
                
                
        geo_level_1_fact=math.pow(10,int(math.log(self._sns_data['geo_level_2_id'].max(),10)+1))
        geo_level_2_fact=math.pow(10,int(math.log(self._sns_data['geo_level_3_id'].max(),10)+1))
        self._sns_data['geo_level_n']=  self._sns_data['geo_level_1_id']*geo_level_1_fact*geo_level_2_fact+self._sns_data['geo_level_2_id']*geo_level_2_fact+self._sns_data['geo_level_3_id']
        self._sns_data['geo_level']=self._sns_data['geo_level_n']#.astype(np.int64).astype(str)
        self._sns_data['geo_level_2']=self._sns_data['geo_level_n']*self._sns_data['geo_level_n']
        self._sns_data['geo_level_cod']=self._sns_data['geo_level_n'].astype(np.int64)

        self._sns_data=self._sns_data.drop(['geo_level_n','geo_level'],axis=1)

        self._sns_data=self._sns_data.drop(['count_floors_pre_eq'],axis=1)
        min_max_scaler = MinMaxScaler()
        self._sns_data[list(self._sns_data.columns)]=min_max_scaler.fit_transform(self._sns_data.values)
        lcolum_x = list(self._sns_data.columns) 
        print("NEcoder 1",self._sns_data.shape)
        print('Fin transform')
        return self._sns_data
    
    def fit(self,X, y=None, **kwargs):
        for key, value in kwargs.items():
            if key=="caso":
                self._caso=value
        s = (X.dtypes == 'object')
        self.object_cols = list(s[s].index)
        s = (X.dtypes != 'object')        
        self.number_cols = list(s[s].index) 
        self._y=y
        self.inicializamodelo(self._caso)
        self.transform(X,y, **kwargs)
        print('Fin fit')
        return self

    def predict(self, X):
        return(self._y)

    def fit_transform(self,X, y=None, **kwargs):
            miX=X.values
            miX=miX.reshape(-1, 1)
            self.fit(X, y, **kwargs)
            nada=self.transform(X, **kwargs)
            print('Fin fit_transform')
            return(nada)
        
    def set_params(self,**kwargs):
            for key, value in kwargs.items():
                if key=="caso":
                    self._caso=value
                    print(key,self._caso)
            return self        
    

In [92]:
def cargamodelo(modelcaso):
    global model
    if modelcaso==1:
        model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',max_iter=500)
    if modelcaso==2:    
        model = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=100)
    if modelcaso==3:
        model = DecisionTreeClassifier(random_state=0,max_depth=20) 
    if modelcaso==4:
        model = MultinomialNB
    if modelcaso==5:
        model = DecisionTreeRegressor(random_state=0)   
    if modelcaso==6:
        model = SVC(gamma='auto',verbose=True,kernel='linear', probability=True)          
    if modelcaso==9:    
        model = Sequential()
        model.add(Dense(12, input_dim=36, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(15, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      

In [93]:
if 'steps' in globals():
    del steps
if 'pipeline' in globals():    
    del pipeline
steps = [('Preprosessor', presotere(caso=1)),('Modelo',RandomForestRegressor(max_depth=20, random_state=0,n_estimators=100))]
#steps = [('Preprosessor', presotere(caso=1))]
pipeline = Pipeline(steps)

In [94]:
parameteres = {'Preprosessor__caso':[1,2],'Modelo__max_depth':[10,20]}
#grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)

In [ ]:
pipeline.fit(train_values,train_labels,Preprosessor__caso=2)

Inicializa OneHotEncoder caso 2
Transforma caso 2
Transforma caso 2.1
Transforma caso 2.2
Transforma caso 2.3 (260601, 38) 38
Transforma caso 2.4 (260601, 38)
NEcoder (260601, 68)
NEcoder 1 (260601, 69)
Fin transform
Fin fit
Transforma caso 2
Transforma caso 2.1
Transforma caso 2.2
Transforma caso 2.3 (260601, 38) 38
Transforma caso 2.4 (260601, 38)
NEcoder (260601, 68)
NEcoder 1 (260601, 69)
Fin transform
Fin fit_transform


c:\program files\python37\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [ ]:
grid.fit(train_values,train_labels,scoring="f1_micro" ,n_jobs=1 )

In [ ]:
pipeline.fit(train_values,train_labels)
nada=pipeline.predict(train_values)
test_data=pipeline.predict(test_values)
pvalues=test_values[['geo_level_1_id']]
pvalues['damage_grade']=test_data.round().astype(np.int64)
pvalues=pvalues.drop(['geo_level_1_id'],axis=1)
pvalues.to_csv(DATA_DIR / 'submission_00_05.csv')

In [56]:
print(train_values.shape)

(260601, 38)
